# Extraction

## Rotowire

In [1]:
import datasets
data = datasets.load_dataset('GEM/RotoWire_English-German')

README.md:   0%|          | 0.00/23.0k [00:00<?, ?B/s]

RotoWire_English-German.py:   0%|          | 0.00/23.5k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.27M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/242 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/241 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/240 [00:00<?, ? examples/s]

In [10]:
ex = data["test"].to_pandas().iloc[4]

In [15]:
for key in ex.keys():
    print(f"{key}: {ex[key]}")
    print("-"*10)

id: 03_17_15-Pelicans-Bucks-TheNewOrleansPelicans(
----------
gem_id: GEM-RotoWire_English-German-test-4
----------
home_name: Pelicans
----------
box_score: {'FIRST_NAME': {'0': 'Giannis', '1': 'Ersan', '2': 'Zaza', '3': 'Khris', '4': 'Michael', '5': 'Jerryd', '6': 'Chris', '7': 'Tyler', '8': 'John', '9': 'Jared', '10': 'O.J.', '11': 'Johnny', '12': 'Miles', '13': 'Quincy', '14': 'Anthony', '15': 'Omer', '16': 'Eric', '17': 'Tyreke', '18': 'Alexis', '19': 'Norris', '20': 'Dante', '21': 'Elliot', '22': 'Ryan', '23': 'Luke', '24': 'Jimmer', '25': 'Jeff'}, 'MIN': {'0': '36', '1': '36', '2': '24', '3': '35', '4': '25', '5': '34', '6': '3', '7': '23', '8': '24', '9': 'N/A', '10': 'N/A', '11': 'N/A', '12': 'N/A', '13': '38', '14': '42', '15': '29', '16': '35', '17': '37', '18': '17', '19': '20', '20': '14', '21': '8', '22': 'N/A', '23': 'N/A', '24': 'N/A', '25': 'N/A'}, 'FGM': {'0': '6', '1': '6', '2': '3', '3': '2', '4': '2', '5': '4', '6': '0', '7': '2', '8': '7', '9': 'N/A', '10': 'N/A',

### Using struc bench directly

In [ ]:
import json

with open("data/struc_bench/table_train.json", "r", encoding="utf-8") as f:
    table_train = json.load(f)

table_train

[{'instruction': 'Here I want you to generate a table based on the given input. Below I first give you text to describe the information in the table you need to generate. Then I give you the format of the table you need to generate, such as column names and the format of cell elements. Now you will generate a table based on the input: ',
  'input': 'Input text: The Phoenix Suns (39 - 38) escaped with an 87 - 85 win over the Utah Jazz (34 - 42). Rodney Hood missed the game - winning three - pointer at the buzzer. He was potentially fouled, but the official ruled otherwise. The Suns moved to three and a half games behind the Oklahoma City Thunder for the final playoff spot with five games remaining. Gerald Green scored 24 points in just 22 minutes for the first time since November 26th. After three straight double - figure scoring games, rookie T.J. Warren was limited to four points with Green putting on a show. Brandan Wright continues to start for the injured Alex Len, supplying 14 poi

In [30]:
import re
import json

def to_snake_case(s: str) -> str:
    return re.sub(r'\W+', '_', s.strip().lower()).strip('_')

def try_parse_number(s: str):
    if s == '':
        return None
    try:
        return int(s)
    except ValueError:
        try:
            return float(s)
        except ValueError:
            return s

def parse_table(table_text: str, id_field: str) -> list:
    lines = [line.strip() for line in table_text.splitlines() if line.strip().startswith('|')]
    if not lines:
        return []
    
    header_fields = [h.strip() for h in lines[0].strip('|').split('|')]
    headers = [
        id_field if (not h or h.isspace()) else to_snake_case(h)
        for h in header_fields
    ]
    
    records = []
    for line in lines[1:]:
        values = [v.strip() for v in line.strip('|').split('|')]
        if len(values) < len(headers):
            values += [''] * (len(headers) - len(values))
        values = values[:len(headers)]
        record = {
            key: try_parse_number(val)
            for key, val in zip(headers, values)
        }
        records.append(record)
    
    return records

def parse_text(text: str) -> list:
    text = text.replace("<NEWLINE>", "\n")
    sections = re.findall(r'Team:\s*(.*?)\s*(?=Team:|$)', text, flags=re.S)
    result = []
    for section in sections:
        parts = re.split(r'Player:\s*', section, maxsplit=1)
        team_part = parts[0]
        player_part = parts[1] if len(parts) > 1 else ''
        teams = parse_table(team_part, 'team')
        players = parse_table(player_part, 'player')
        result.append({
            'teams': teams,
            'players': players
        })
    return result

# Example usage on a subset of your data (first section):
sample = """
Team:
|  | Losses | Total points | Points in 4th quarter | Wins |
| Hawks | 12 | 95 |  | 46 |
| Magic | 41 | 88 | 21 | 19 |
Player:
|  | Assists | Points | Total rebounds | Steals |
| Nikola Vucevic |  | 21 | 15 |  |
| Al Horford | 4 | 17 | 13 | 2 |
| Jeff Teague | 7 | 17 |  | 2 |
"""

ex = """Team: <NEWLINE> |  | Percentage of field goals | Losses | Total points | Points in 4th quarter | Wins | <NEWLINE> | Raptors | 53 | 15 | 114 | 25 | 29 | <NEWLINE> | Pistons | 46 | 28 | 110 | 31 | 17 | <NEWLINE> Player: <NEWLINE> |  | Assists | Defensive rebounds | 3-pointers attempted | 3-pointers made | Field goals attempted | Field goals made | Free throws attempted | Free throws made | Minutes played | Personal fouls | Points | Total rebounds | Steals | <NEWLINE> | Greg Monroe |  |  |  |  | 17 | 9 | 5 | 3 |  |  | 21 | 16 |  | <NEWLINE> | Andre Drummond |  |  |  |  | 11 | 7 | 1 | 0 |  |  | 14 | 8 |  | <NEWLINE> | Kentavious Caldwell-Pope |  |  | 9 | 3 | 17 | 6 | 2 | 1 |  |  | 16 | 3 | 2 | <NEWLINE> | D.J. Augustin | 8 |  | 9 | 5 | 20 | 12 |  |  |  |  | 35 |  |  | <NEWLINE> | DeMar DeRozan | 4 |  | 2 | 1 | 14 | 8 | 10 | 8 |  |  | 25 | 6 |  | <NEWLINE> | Amir Johnson |  |  |  |  | 9 | 7 | 4 | 3 |  |  | 17 | 2 |  | <NEWLINE> | Greivis Vasquez |  |  |  |  |  |  |  |  |  |  | 13 |  |  | <NEWLINE> | Lou Williams |  |  |  |  |  |  |  |  |  |  | 12 |  |  | <NEWLINE> | Jonas Valanciunas |  | 3 |  |  | 15 | 9 | 2 | 2 |  | 3 | 20 | 11 |  |"""
parse_text(ex)



[{'teams': [{'team': 'Raptors',
    'percentage_of_field_goals': 53,
    'losses': 15,
    'total_points': 114,
    'points_in_4th_quarter': 25,
    'wins': 29},
   {'team': 'Pistons',
    'percentage_of_field_goals': 46,
    'losses': 28,
    'total_points': 110,
    'points_in_4th_quarter': 31,
    'wins': 17}],
  'players': [{'player': 'Greg Monroe',
    'assists': None,
    'defensive_rebounds': None,
    '3_pointers_attempted': None,
    '3_pointers_made': None,
    'field_goals_attempted': 17,
    'field_goals_made': 9,
    'free_throws_attempted': 5,
    'free_throws_made': 3,
    'minutes_played': None,
    'personal_fouls': None,
    'points': 21,
    'total_rebounds': 16,
    'steals': None},
   {'player': 'Andre Drummond',
    'assists': None,
    'defensive_rebounds': None,
    '3_pointers_attempted': None,
    '3_pointers_made': None,
    'field_goals_attempted': 11,
    'field_goals_made': 7,
    'free_throws_attempted': 1,
    'free_throws_made': 0,
    'minutes_played':

In [31]:
import numpy as np
subsamples = np.random.choice(table_train, size=5, replace=False)

for i, sample in enumerate(subsamples):
    print(f"Sample {i+1}:")
    print("Text:", sample["input"])
    print("Parsed:", parse_text(sample["output"]))
    print("Original:", sample["output"])
    print("-" * 40)

Sample 1:
Text: Input text: The Minnesota Timberwolves defeated the Charlotte Hornets in overtime, 125 - 120, at Spectrum Center on Saturday. The Timberwolves (6 - 14) may have looked like somewhat easy prey to the hometown Hornets, considering they came in with a 5 - 14 record despite their solid talent. The first quarter did nothing to dissuade those beliefs, as Charlotte (11 - 9) jumped out to a 29 - 19 lead after the first 12 minutes. However, Minnesota clawed back to trail by only three, 52 - 49 by intermission. The second half was a continuous see - saw battle that appeared like it might go the Hornets' way in the closing seconds, until an Andrew Wiggins 31 - foot three - point jumper with eight seconds remaining knotted the score at 106. He then sunk a pair of clutch free throws with 11 seconds remaining in overtime to give Minnesota some crucial breathing room at 123 - 118. That five - point margin was the final separation between the two teams, as the Timberwolves secured only

In [ ]:
parsed = []

for i, sample in enumerate(table_train):
    obj = {
        "input":   sample["input"],
        "output":  parse_text(sample["output"]),
    }
    parsed.append(obj)

with open("data/clean/struc_bench/parsed.json", "w", encoding="utf-8") as f:
    json.dump(parsed, f, indent=4)

## Wikibio

In [12]:
from datasets import load_dataset

#data = load_dataset("michaelauli/wiki_bio", trust_remote_code=True, split="test")
#data.to_json("data/clean/wiki_bio/raw.json", orient="records", lines=True)

with open("data/clean/wiki_bio/raw.json", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f.readlines()]

In [14]:
t = data[14]
print("input text:")
print(t["input_text"])
print("----")
print("target text:")
print(t["target_text"])

input text:
{'table': {'column_header': ['birth_place', 'article_title', 'name'], 'row_number': [1, 1, 1], 'content': ['belfast , northern ireland', 'peter curran -lrb- presenter -rrb-\n', 'peter curran']}, 'context': 'peter curran -lrb- presenter -rrb-\n'}
----
target text:
peter curran is an irish broadcaster , writer , documentary maker and audiobook publisher .
he grew up in belfast , the eldest of six children .
after leaving school he worked on funfairs in the usa before moving to london .
curran worked for an interior design and building company for four years before re-training as a bbc reporter .
in 1992 , he began dj ` ing full-time for the london radio station bbc glr .



In [ ]:
# clean tokenization

import re

PTB_PARENS = {
    "-LRB-": "(", "-RRB-": ")",
    "-LSB-": "[", "-RSB-": "]",
    "-LCB-": "{", "-RCB-": "}"
}

def detokenize_ptb(text):
    for k, v in PTB_PARENS.items():
        text = text.replace(k, v)
    # Collapse spaces before commas and periods added by PTB
    text = re.sub(r"\s+([,.;:?!)])", r"\1", text)
    text = re.sub(r"([(\[{])\s+", r"\1", text)
    return text.strip()

detokenize_ptb(t["target_text"])

'peter curran is an irish broadcaster, writer, documentary maker and audiobook publisher.\nhe grew up in belfast, the eldest of six children.\nafter leaving school he worked on funfairs in the usa before moving to london.\ncurran worked for an interior design and building company for four years before re-training as a bbc reporter.\nin 1992, he began dj ` ing full-time for the london radio station bbc glr.'

In [16]:
data[37]["input_text"]

{'table': {'column_header': ['appointed',
   'successor',
   'article_title',
   'predecessor',
   'religion',
   'see',
   'title',
   'death_date',
   'name',
   'consecration',
   'ended',
   'consecrated_by'],
  'row_number': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'content': ['before 733',
   'aluberht',
   'sigeferth of selsey\n',
   'eolla',
   'catholic',
   'diocese of selsey',
   'bishop of selsey',
   'between 747 and 765',
   'sigeferth',
   '733',
   'between 747 and 765',
   'tatwine']},
 'context': 'sigeferth of selsey\n'}

In [57]:
# Full pipeline 

from dateutil import parser as dt         
from datetime import datetime
import json, re
from datasets import load_dataset

PTB = {
    "-LRB-": "(", "-RRB-": ")", "-LSB-": "[", "-RSB-": "]",
    "-LCB-": "{", "-RCB-": "}",
}

# Pre-compile a *case-insensitive* pattern that hits every PTB code
PTB_PATTERN = re.compile(r"-(lrb|rrb|lsb|rsb|lcb|rcb)-", flags=re.I)

SPACE_BEFORE = re.compile(r"\s+([,.;:?!)'])")  # now includes the apostrophe
SPACE_AFTER  = re.compile(r"([(\[{])\s+")

def detok(text: str) -> str:
    """Case-insensitive PTB detokeniser + whitespace fixer."""
    # Replace tokens with their PTB equivalents
    text = PTB_PATTERN.sub(lambda m: PTB[m.group(0).upper()], text)

    # Collapse multiple spaces
    text = SPACE_BEFORE.sub(r"\1", text)
    text = SPACE_AFTER.sub(r"\1", text)

    # Collapse spaces around apostrophes
    text = re.sub(r"\s+'\s*", "'", text)

    return text.strip()

DEFAULT = datetime(1, 1, 1)          

def normalize_date(raw: str) -> str:
    """
    Convert free-form dates to YYYY-MM-DD.
    • Full dates → '1949-02-12'
    • Year only  → '1949'
    • No year    → original text  ('June 16')
    • Ranges / fuzzy strings → original text
    """
    # Strip ordinal suffixes: "12th" → "12"
    clean = re.sub(r"(\d+)(st|nd|rd|th)", r"\1", raw, flags=re.I)

    # Reject obvious ranges before parsing
    if re.search(r"\bbetween\b|\band\b|\bto\b|–|-", clean) and re.search(r"\d", clean):
        return raw

    try:
        dt_obj = dt.parse(clean, default=DEFAULT, fuzzy=True)
    except (ValueError, OverflowError):
        return raw                               # Unparseable

    if dt_obj.year == DEFAULT.year:
        return raw                               # Year was missing
    if dt_obj.month == 1 and dt_obj.day == 1 and re.fullmatch(r"\d{4}", clean.strip()):
        return f"{dt_obj.year}"                  # Year only
    return dt_obj.strftime("%Y-%m-%d")


def row_to_dict(example: dict) -> dict:
    cols = example["table"]["column_header"]
    vals = example["table"]["content"]

    record = {}
    for col, v in zip(cols, vals):
        text = detok(v)
        if "date" in col.lower():
            text = normalize_date(text)
        record[col] = text

    return record

In [58]:
row_to_dict(data[89]["input_text"])

{'office': 'prime minister of nepal president of the nepali congress',
 'term_start': '11 february 2014 22 september 2010',
 'article_title': 'sushil koirala',
 'image': 'sushil koirala.jpg',
 'president': 'ram baran yadav',
 'predecessor': 'khil raj regmi girija prasad koirala',
 'party': 'nepali congress',
 'religion': 'hinduism',
 'name': 'sushil koirala. सुशील कोइराला',
 'birth_place': 'biratnagar',
 'birth_date': '1939-08-12'}

In [59]:
row_to_dict(data[224]["input_text"])

{'death_cause': 'guillain-barré syndrome',
 'party': 'republican',
 'name': 'norton simon',
 'death_place': 'beverly hills, california',
 'spouse': 'lucille jones (1971-1993; his death) ellis (? -1970; divorced) jennifer',
 'religion': 'judaism',
 'residence': 'beverly hills, california',
 'birth_name': 'norton winfred simon',
 'birth_date': '1907-02-05',
 'citizenship': 'united states',
 'birth_place': 'portland, oregon',
 'alma_mater': 'university of california, berkeley',
 'occupation': 'industrialist, philanthropist',
 'article_title': 'norton simon',
 'known_for': "founder of hunt's foods of the eponymous norton simon museum, founder",
 'home_town': 'fullerton, california',
 'children': 'robert simon (deceased)',
 'death_date': '1993-06-02',
 'boards': "hunt's foods"}

In [60]:
detok(data[224]["target_text"])

'norton winfred simon (february 5, 1907 -- june 2, 1993), in the united states was a millionaire industrialist and philanthropist based in california.\nhis significant art collection is housed in the norton simon museum in pasadena, california.'

In [61]:
data_clean = []

for i, sample in enumerate(data):
    obj = {
        "input": detok(sample["target_text"]),
        "output": row_to_dict(sample["input_text"]),
    }
    data_clean.append(obj)

In [63]:
with open("data/clean/wiki_bio/parsed.json", "w", encoding="utf-8") as f:
    json.dump(data_clean, f, indent=4)